In [4]:
import os
import json
from pathlib import Path
from PIL import Image

# 📌 Configuración de clases (debe coincidir con tu data.yaml de YOLO)
categories = [
    {"id": 1, "name": "person"},
    # Agrega más clases según corresponda
]

# 📂 Carpeta base del dataset YOLO
base_dir = Path("../data/yolo")
output_dir = Path("../data/coco")
splits = ["train", "val"]  # conjuntos a procesar

os.makedirs(output_dir, exist_ok=True)

def yolo_to_coco(images_dir, labels_dir, output_json):
    """
    Convierte anotaciones YOLO a formato COCO.
    """
    coco_format = {
        "images": [],
        "annotations": [],
        "categories": categories
    }

    annotation_id = 0
    for img_id, img_file in enumerate(sorted(images_dir.glob("*.png"))):
        with Image.open(img_file) as im:
            width, height = im.size

        coco_format["images"].append({
            "id": img_id,
            "file_name": img_file.name,
            "width": width,
            "height": height
        })

        label_file = labels_dir / f"{img_file.stem}.txt"
        if not label_file.exists():
            continue

        with open(label_file, "r") as lf:
            for line in lf:
                cls, x_center, y_center, w, h = map(float, line.strip().split())

                x_min = (x_center - w/2) * width
                y_min = (y_center - h/2) * height
                w_px = w * width
                h_px = h * height

                coco_format["annotations"].append({
                    "id": annotation_id,
                    "image_id": img_id,
                    "category_id": int(cls),
                    "bbox": [x_min, y_min, w_px, h_px],
                    "area": w_px * h_px,
                    "iscrowd": 0
                })
                annotation_id += 1


    with open(output_json, "w") as f:
        json.dump(coco_format, f, indent=4)

    print(f"COCO JSON guardado en: {output_json}")
 

# 🔄 Procesar train y val
for split in splits:
    images_dir = base_dir / "images" / split
    labels_dir = base_dir / "labels" / split
    output_json = output_dir / f"coco_{split}.json"

    yolo_to_coco(images_dir, labels_dir, output_json)


COCO JSON guardado en: ../data/coco/coco_train.json
COCO JSON guardado en: ../data/coco/coco_val.json


### Split dir on bash

In [ ]:
!pip install pycocotools
python3 yolo_to_coco.py --yolo_dir ./data --output_dir ./coco_annotations